# Core-to-log matching

In [ ]:
import os
import sys

sys.path.insert(0, os.path.join("..", "..", ".."))
from petroflow import WellDataset
from petroflow.batchflow import Pipeline, C
from petroflow.batchflow.research import Research, Option

In [ ]:
DATASET_PATH = "/Raw_dataset/*"
well_ds = WellDataset(path=DATASET_PATH, dirs=True, sort=True)

In [ ]:
MATCHED_DATASETS_PATH = "/Matched_dataset/"
WIN_SIZES = [None, 51, 101, 151, 201, 251, 301]

grid = Option.product(
    Option("win_size", WIN_SIZES),
    Option("dump_path", [os.path.join(MATCHED_DATASETS_PATH, str(win_size)) for win_size in WIN_SIZES])
)

In [ ]:
matching_modes = [
    "GK ~ core_logs.GK",
    "DENSITY ~ core_logs.DENSITY",
    "DENSITY ~ core_properties.DENSITY",
    "DENSITY ~ core_properties.POROSITY",
    "DT ~ core_properties.POROSITY",
    "NKTD ~ core_properties.POROSITY",
]

In [ ]:
matching_pipeline = (Pipeline()
    .check_regularity()
    .match_core_logs(mode=matching_modes, split_lithology_intervals=False,
                     gaussian_win_size=C("win_size"), save_report=False)
    .dump(C("dump_path"))
    .run(batch_size=4, n_epochs=1, shuffle=False, drop_last=False, lazy=True)
)

In [ ]:
research = (Research()
    .add_pipeline(matching_pipeline, dataset=well_ds, name="matching")
    .add_grid(grid)
)

research.run(name="matching_research", bar=True)